In [1]:
def laplace_multiplicand(filename):
    import skimage
    from skimage import io
    from skimage.transform import resize
    import numpy as np
    from matplotlib import pyplot as plt
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    from skimage.filters import laplace
    %matplotlib inline
    import netCDF4 as NC
    fileID = NC.Dataset(filename)
    
    fires = io.imread('fileID');
    firstChannel = np.squeeze(practice[:,:,0])
    height, width = firstChannel.shape
    smallFires = resize(firstChannel, (round(height/2), round(width/2)))*255
    blurryFires = resize(smallFires, (height, width))
    laplace_edges = laplace(blurryFires)
    
    file = open('multiplicand.csv','a')
        file.write('File Name, Mean Absolute Error')
        file.close()
    
    multiplicand = 0
    while multiplicand <= 1
        sharped_image = blurryFires+laplace_edges * multiplicand
        abs_error = abs(firstChannel-sharped_image)
        sqrt = np.sqrt(((details- laplace_edges)**2).mean())
        file = open('multiplicand.csv','a')
        file.write('fileID,multiplicand,abs_error.mean()')
        file.close()
        multiplicand = multiplicand + .1
        return    
    
    ##posi_file = find(File Name('fileID'))
    ##posi_min = 